# Data Integration
## Authors: 
    1. Lam Ho
    2. Jonah Breslow
    3. Jeffrey Kagan
## Purpose:
The purpose of this notebook is to ingest all the matched entities across the three datasets and generate a set of nodes and edges that will populate our Neo4j knowledge graph.

### Importing modules

In [1]:
import pandas as pd
import numpy as np
import pickle

## Importing Entity Resolution Lookup tables
### Manufacturer Node

In [2]:
manuf_lookup = pickle.load(open('Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.p','rb'))
manuf_lookup[['manuf_id','manufacturer_name']].to_csv('Nodes/Manufacturer_Node.csv',index=False)
print("Manufacturer Node has "+str(manuf_lookup.shape[0])+" records")

Manufacturer Node has 10621 records


### Drug Node

In [3]:
def string_to_ordered_list (x):
    '''
    Function that takes a string that looks like a list, and converts it to an ordered list
    '''
    ordered_list = None
    try: 
        str_list = str(x).replace('[','').replace(']','').replace(' ','').replace("'",'').split(',')
        str_list = list(map(int,str_list))
        str_set = set(str_list)
        ordered_list = sorted(list(str_set))
    except:
        ordered_list = np.NaN
    
    
    return ordered_list

In [4]:
## Importing drug matching output
drug_node = pickle.load(open('Data/Outputs_Cleanup/Sunshine_dedupe_drug_integration/final_drug_lookup.p','rb'))

## Importing drug information from the deduped FDA data
openfda_drug_dedupe = pickle.load(open( "Data/Outputs_Cleanup/FDA/Openfda_Drug_deduplicated.p", "rb" ))

## Sorting the fda_drug_id list column so that when we convert to a string, it will 
## match the fda_drug_id list column in the drug_node dataframe
openfda_drug_dedupe = openfda_drug_dedupe.applymap(set).applymap(list)
openfda_drug_dedupe.fda_drug_id = openfda_drug_dedupe.fda_drug_id.apply(sorted)

## converting the fda_drug_id string column into a sorted list column
drug_node['fda_drug_id'] = drug_node.fda_drug_id.apply(string_to_ordered_list)

## converting fda_drug_id columns in drug_node df and openfda_drug_dedupe df 
## to string from list for merging
drug_node['fda_drug_id'] = drug_node['fda_drug_id'].astype(str)
openfda_drug_dedupe['fda_drug_id'] = openfda_drug_dedupe['fda_drug_id'].astype(str)
drug_node = drug_node.merge(openfda_drug_dedupe,'left','fda_drug_id')

## Dropping id's into other datasources
# drug_node
drug_node_output = drug_node.drop(columns = ['fda_drug_id','MedD_drug_id','sunshine_drug_id','manufacturer_name'])
drug_node_output.to_csv('Nodes/Drug_Node.csv',index=False)
print("Drug Node has "+str(drug_node_output.shape[0])+" records")

Drug Node has 5116 records


## Starting with the Sunshine Data
### Hospital Node

In [5]:
# hospital_node = pd.read_csv('Data/Outputs_Cleanup/Sunshine/hosp_info.csv')
# hospital_node.to_csv('Nodes/Hospital_Node.csv',index=False)
# print("Hospital Node has "+str(hospital_node.shape[0])+" records")

### Payments Node

In [6]:
payments_node = pd.read_csv('Data/Outputs_Cleanup/Sunshine/payments.csv')
payments_node.to_csv('Nodes/Payments_Node.csv',index=False)
print("Payments Node has "+str(payments_node.shape[0])+" records")

Payments Node has 1094501 records


### Manufacturer-Payments Edge

In [7]:
manuf_payments_edge = pd.read_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv')
manuf_payments_edge = manuf_payments_edge.merge(manuf_lookup,'left','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID')
manuf_payments_edge = manuf_payments_edge[['Record_ID','manuf_id']]
manuf_payments_edge.to_csv('Edges/Manufacturer_Payment_Edge.csv',index=False)
print("manuf_payments_edge has "+str(manuf_payments_edge.shape[0])+" records")

manuf_payments_edge has 1094501 records


### Payment-Drug Edge

In [8]:
payment_drug_edge = pd.read_csv('Data/Outputs_Cleanup/Sunshine/record_drugs.csv')
cols = ['Record_ID','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5'
       ]
payment_drug_edge = payment_drug_edge[cols].melt(id_vars = 'Record_ID')
payment_drug_edge.dropna(inplace=True)
payment_drug_edge = payment_drug_edge[['Record_ID','value']]
payment_drug_edge.columns = ['Record_ID','sunshine_drug_id']
drug_node[['drug_id','sunshine_drug_id']].dropna()
payment_drug_edge = payment_drug_edge.merge(drug_node[['drug_id','sunshine_drug_id']].dropna(),
                                           'left',
                                           'sunshine_drug_id')
payment_drug_edge = payment_drug_edge[~payment_drug_edge.drug_id.isna()][['Record_ID','drug_id']]

payment_drug_edge.drug_id= payment_drug_edge.drug_id.astype(int)
payment_drug_edge.to_csv('Edges/Payment_Drug_Edge.csv',index=False)
print("payment_drug_edge has "+str(payment_drug_edge.shape[0])+" records")

C:\Users\10294029\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (17,18,19,20,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


payment_drug_edge has 97474 records


### Manufacturer-Drug edge

In [9]:
openfda_manuf_dedupe = pickle.load(open( "Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.p", "rb" ))

manuf_drug_edge = drug_node[['drug_id','manufacturer_name']]
manuf_drug_edge = manuf_drug_edge.explode('manufacturer_name')
manuf_drug_edge.dropna(inplace=True)
manuf_drug_edge = manuf_drug_edge.merge(openfda_manuf_dedupe.explode('manuf_names'),'left',left_on='manufacturer_name',right_on='manuf_names')
# manuf_drug_edge
# manuf_lookup
manuf_drug_edge = manuf_drug_edge[['drug_id','fda_manuf_id','manufacturer_name_y']].explode('fda_manuf_id')
manuf_drug_edge = manuf_drug_edge.drop_duplicates(['drug_id','manufacturer_name_y'])[['drug_id','fda_manuf_id']]
manuf_drug_edge = manuf_drug_edge.merge(manuf_lookup.explode('fda_manuf_id'),'left','fda_manuf_id')
manuf_drug_edge = manuf_drug_edge[['drug_id','manuf_id']]
manuf_drug_edge.to_csv('Edges/Manufacturer_Drug_Edge.csv',index=False)
print("manuf_drug_edge has "+str(manuf_drug_edge.shape[0])+" records")

manuf_drug_edge has 12837 records


### Physicians Node

In [10]:
physicians = pickle.load(open("Data/Physicians_Deduplication/Outputs/Physicians.pkl", "rb" ))
physicians.rename(columns={'id':'physician_id'},inplace=True)
physician_info = physicians.drop(columns=['Prscrbr_NPI','Physician_Profile_ID'])
physician_info.to_csv('Nodes/Physicians_Node.csv',index=False)
print("Physicians Node has "+str(physician_info.shape[0])+" records")

Physicians Node has 137176 records


### Prescription Node, Prescription-Drug Edge, Prescription-Physician Edge

In [11]:
prescriptions = pickle.load(open("Data/Outputs_Cleanup/Part_d/prescription_information.p", "rb" ))
prescriptions.head()

partD_to_drug_id = drug_node[['drug_id','MedD_drug_id']]
partD_to_drug_id['MedD_drug_id'] = partD_to_drug_id['MedD_drug_id'].apply(string_to_ordered_list)
partD_to_drug_id = partD_to_drug_id.explode('MedD_drug_id')
partD_to_drug_id.MedD_drug_id = partD_to_drug_id.MedD_drug_id.astype(float)

prescriptions = prescriptions.merge(partD_to_drug_id,'left','MedD_drug_id')
prescriptions.drop(columns=['GE65_Tot_Clms','GE65_Tot_30day_Fills','GE65_Tot_Drug_Cst','GE65_Tot_Day_Suply','GE65_Tot_Benes'],inplace=True)
physician_Part_d_lookup = physicians.explode('Prscrbr_NPI').drop(columns=['Physician_Profile_ID','fname','lname','type','city'])
physician_Part_d_lookup.dropna(inplace=True)
physician_Part_d_lookup['Prscrbr_NPI'] = physician_Part_d_lookup['Prscrbr_NPI'].astype(int)
prescriptions = prescriptions.merge(physician_Part_d_lookup,'left','Prscrbr_NPI')
prescriptions.drop(columns=['Prscrbr_NPI','MedD_drug_id'],inplace=True)
prescriptions = prescriptions.groupby(['drug_id', 'physician_id','Year']).sum().reset_index().reset_index()
prescriptions.rename(columns={'index':'prescription_id'},inplace=True)

<ipython-input-11-5cdb63e96030>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partD_to_drug_id['MedD_drug_id'] = partD_to_drug_id['MedD_drug_id'].apply(string_to_ordered_list)


In [12]:
prescription_drug_edge = prescriptions.drop(columns=['physician_id','Year','Tot_Clms',
                                                     'Tot_30day_Fills','Tot_Day_Suply',
                                                     'Tot_Drug_Cst','Tot_Benes'])
prescription_physician_edge = prescriptions.drop(columns=['drug_id','Year','Tot_Clms',
                                                          'Tot_30day_Fills','Tot_Day_Suply',
                                                          'Tot_Drug_Cst','Tot_Benes'])
prescriptions.drop(columns=['drug_id', 'physician_id'],inplace=True)

In [13]:
prescription_drug_edge.to_csv('Edges/Prescription_Drug_Edge.csv',index=False)
print("prescription_drug_edge has "+str(prescription_drug_edge.shape[0])+" records")
prescription_physician_edge.to_csv('Edges/Prescription_Physician_Edge.csv',index=False)
print("prescription_physician_edge has "+str(prescription_physician_edge.shape[0])+" records")
prescriptions.to_csv('Nodes/Prescriptions_Node.csv',index=False)
print("prescriptions Node has "+str(prescriptions.shape[0])+" records")

prescription_drug_edge has 2415170 records


prescription_physician_edge has 2415170 records


prescriptions Node has 2415170 records


In [14]:
payments_physician = pd.read_csv('Data/Outputs_Cleanup/Sunshine/physicians_payments.csv',dtype=int)
physicians_sunshine_lookup = physicians.explode('Physician_Profile_ID').drop(columns=['Prscrbr_NPI','fname','lname','type','city'])
physicians_sunshine_lookup.dropna(inplace=True)
physicians_sunshine_lookup['Physician_Profile_ID']=physicians_sunshine_lookup['Physician_Profile_ID'].astype(int)
payments_physician = payments_physician.merge(right=physicians_sunshine_lookup,how='left',on='Physician_Profile_ID')
payments_physician.drop(columns='Physician_Profile_ID',inplace=True)
payments_physician.to_csv('Edges/Payments_Physician_Edge.csv',index=False)
print("payments_physician has "+str(payments_physician.shape[0])+" records")

payments_physician has 1088993 records
